In [1]:
import pymupdf
invoice_file = "./invoices/4.pdf"

doc = pymupdf.open(invoice_file)
texts = []
for page in doc:
    text = page.get_text("text")
    texts.append(text)
    # print(text)


In [2]:
import dotenv

dotenv.load_dotenv()


True

In [3]:
create_prompt = lambda txt : f"""
You are a helpful assistant for automating data entry into an accounting system.
Given the text extracted from an invoice, you need to extract the following information:

for the invoice get
Customer name, Supplier name, document number, document date

for each good in the invoice, get the 
code, Product Name / Recognized Product, Unit, Quantity, Price, Taxable Amount, Tax Rate, Tax, Total Amount

Keep the output format strictly as follows:
line 1 for invoice titles in csv format
line 2 for invoice data in csv format
line 3 for goods title in csv format
line 4 onwards for goods data in csv format
Output no other information.

The invoice text is as follows:
{txt}
"""


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0.3)


In [5]:
prompt = create_prompt(texts[0])

msg = llm.invoke(prompt)
msg.content

# print(msg.content, end="", flush=True)


'```csv\nCustomer Name,Supplier Name,Document Number,Document Date\nINDIUM SOFTWARE (I) LTD,RAJLAXMI INFOTECH,RI/2019-20/1379,13-Jul-2019\nCode,Product Name / Recognized Product,Unit,Quantity,Price,Taxable Amount,Tax Rate,Tax,Total Amount\n8523,WD SSD 240 GB,NOS,2,2118.64,4237.28,18%,762.71,5000.00\n8523,WD 480GB SSD DRIVE,NOS,1,3771.19,3771.19,18%,678.81,4450.00\n85078000,LAPTOP BATTERY,NOS,1,1900.00,1900.00,18%,342.00,2242.00\n8544,HD CABLE 10M,NOS,1,250.00,250.00,18%,45.00,295.00\n8517,ENTER WIRLESS USB ADPTER,NOS,1,250.00,250.00,18%,45.00,295.00\n```'

In [11]:
import pandas as pd
import io

def process_csv_string(csv_string):
    # Remove the markdown code block syntax
    csv_string = csv_string.replace('```csv\n', '').replace('```', '')
    
    # Split the CSV string into lines
    lines = csv_string.strip().split('\n')
    
    # Process common data
    common_data = dict(zip(lines[0].split(','), lines[1].split(',')))
    common_df = pd.DataFrame([common_data])
    
    # Process item data
    item_header = lines[2].split(',')
    item_data = [line.split(',') for line in lines[3:]]
    item_df = pd.DataFrame(item_data, columns=item_header)
    
    return common_df, item_df

# Your CSV string
msg = '''```csv
Customer Name,Supplier Name,Document Number,Document Date
INDIUM SOFTWARE (I) LTD,RAJLAXMI INFOTECH,RI/2019-20/1379,13-Jul-2019
Code,Product Name / Recognized Product,Unit,Quantity,Price,Taxable Amount,Tax Rate,Tax,Total Amount
8523,WD SSD 240 GB,NOS,2,2118.64,4237.28,18%,762.71,5000.00
8523,WD 480GB SSD DRIVE,NOS,1,3771.19,3771.19,18%,678.81,4450.00
85078000,LAPTOP BATTERY,NOS,1,1900.00,1900.00,18%,342.00,2242.00
8544,HD CABLE 10M,NOS,1,250.00,250.00,18%,45.00,295.00
8517,ENTER WIRLESS USB ADPTER,NOS,1,250.00,250.00,18%,45.00,295.00
```'''

# Process the CSV string
common_df, item_df = process_csv_string(msg)

# Display the results
print("Common Data:")
print(common_df)
print("\nItem Data:")
print(item_df)

Common Data:
             Customer Name      Supplier Name  Document Number Document Date
0  INDIUM SOFTWARE (I) LTD  RAJLAXMI INFOTECH  RI/2019-20/1379   13-Jul-2019

Item Data:
       Code Product Name / Recognized Product Unit Quantity    Price  \
0      8523                     WD SSD 240 GB  NOS        2  2118.64   
1      8523                WD 480GB SSD DRIVE  NOS        1  3771.19   
2  85078000                    LAPTOP BATTERY  NOS        1  1900.00   
3      8544                      HD CABLE 10M  NOS        1   250.00   
4      8517          ENTER WIRLESS USB ADPTER  NOS        1   250.00   

  Taxable Amount Tax Rate     Tax Total Amount  
0        4237.28      18%  762.71      5000.00  
1        3771.19      18%  678.81      4450.00  
2        1900.00      18%  342.00      2242.00  
3         250.00      18%   45.00       295.00  
4         250.00      18%   45.00       295.00  
